# Imports

In [1]:
from typing import List
import json 
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem
from tqdm.notebook import tqdm
import nmslib

[nltk_data] Downloading package stopwords to /home/diveev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data loading

In [2]:
data = pd.read_csv("../data/kp_films.csv")
data = data.dropna().reset_index().drop(columns='index')

In [3]:
data

,kp_id,rating_kp,genres,description
0,12819,7.70,['триллер' 'мелодрама' 'криминал'],"Миллиардер Томас Краун, пресыщенный финансист,..."
1,1260096,-1.00,['семейный' 'для' 'кино' 'ребенок'],Банда саранчи во главе с безжалостным Боссом о...
2,894673,4.40,['вестерн' 'иностранный' 'триллер' 'мистика' '...,"В горах живёт обособленная община, возглавляем..."
3,485364,6.06,['семейный' 'для' 'кино' 'короткометражный' 'р...,"Фильм высмеивает тех, кто понимает трудовую ди..."
4,1289632,6.40,['семейный' 'приключение' 'кино'],"В скучном городке Линдсборо всё меняется, когд..."
...,...,...,...,...
18494,806159,5.31,['семейный' 'аниме' 'приключение' 'фантастика'...,"Сатоши готовится к новому турниру, где парню н..."
18495,625381,6.40,['русский' 'мультфильм' 'для' 'ребенок' 'детск...,На нелепом и забавном мужчине по прозвищу Пара...
18496,4535755,-1.00,['семейный' 'мультфильм' 'иностранный' 'для' '...,Очаровательная сказка о маленьком лягушонке с ...
18497,1111361,6.60,['драма'],Криминальная драма о реальной истории жизни си...


# Data preprocessing

## *Preprocessing genres*

In [4]:
genres = data.genres.to_list()

for i in range(len(genres)):
    genres[i] = genres[i].replace('[', '')
    genres[i] = genres[i].replace(']', '')
    genres[i] = genres[i].replace(',', '')
    genres[i] = genres[i].replace("'", '')
    genres[i] = genres[i].split()

In [5]:
genres_corpus = []

for genres_list in genres:
    genres_corpus += genres_list

genres_corpus = set(genres_corpus)

In [6]:
genres_corpus = genres_corpus.difference(stopwords.words("russian")) #Removing stopwords from genres

In [7]:
for genre in genres_corpus:
    data[genre] = [1 if genre in genres[i] else 0 for i in range(data.shape[0])] #One hot encoding genres

In [8]:
data.head(10)

,kp_id,rating_kp,genres,description,шоу,ток,кино,развлечение,музыкальный,спорт,...,научпоп,мюзикл,ссср,художественный,сериал,природа,фильм,юмор,приключение,комедия
0,12819,7.70,['триллер' 'мелодрама' 'криминал'],"Миллиардер Томас Краун, пресыщенный финансист,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1260096,-1.00,['семейный' 'для' 'кино' 'ребенок'],Банда саранчи во главе с безжалостным Боссом о...,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,894673,4.40,['вестерн' 'иностранный' 'триллер' 'мистика' '...,"В горах живёт обособленная община, возглавляем...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,485364,6.06,['семейный' 'для' 'кино' 'короткометражный' 'р...,"Фильм высмеивает тех, кто понимает трудовую ди...",0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1289632,6.40,['семейный' 'приключение' 'кино'],"В скучном городке Линдсборо всё меняется, когд...",0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,960385,6.30,['комедия'],С самого начала отношений страстные любовники ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,43127,7.30,['драма' 'советский' 'военный' 'ссср'],"Советский художественный фильм, рассказывающий...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,331881,5.30,['ссср' 'советский' 'комедия'],"Если привычная жизнь уже порядком надоела, а м...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
8,259905,-1.00,['русский' 'советский' 'мультфильм' 'для' 'реб...,Очаровательный щенок Боб больше всего любит сл...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,487544,6.83,['русский' 'семейный' 'документальный' 'познав...,"Режиссер Валерий Бабич, автор исторических про...",0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## *Filtering genres*

In [9]:
#Drop the least appearing genres

genre_to_num = {genre : (data[genre] != 0).sum() for genre in genres_corpus if (data[genre] != 0).sum() > 2000}
one_hot_encoded_genres = data[list(genre_to_num.keys())]
descriptions = data[~(one_hot_encoded_genres==0).all(axis=1)].description
one_hot_encoded_genres = one_hot_encoded_genres.loc[~(one_hot_encoded_genres==0).all(axis=1)].reset_index().drop(columns='index')

In [10]:
genre_to_num

{'ребенок': 2361,
 'зарубежный': 4067,
 'мелодрама': 3646,
 'триллер': 2776,
 'семейный': 2209,
 'боевик': 2079,
 'драма': 7091,
 'мультфильм': 2635,
 'русский': 3698,
 'иностранный': 2937,
 'приключение': 2200,
 'комедия': 4494}

In [11]:
one_hot_encoded_genres

,ребенок,зарубежный,мелодрама,триллер,семейный,боевик,драма,мультфильм,русский,иностранный,приключение,комедия
0,0,0,1,1,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,0,1,0,0
3,1,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
17684,0,1,0,0,1,0,0,0,0,0,1,0
17685,1,0,0,0,0,0,0,1,1,0,0,0
17686,1,1,0,0,1,0,0,1,0,1,0,0
17687,0,0,0,0,0,0,1,0,0,0,0,0


## *Preprocessing and encoding descriptions*

In [12]:
def normalize_text(strings:List[str]) -> List[str]:
    normalized = []
    
    for string in tqdm(strings, desc='Normalizing texts'):
        string = re.findall(r"\w+", str(string))
        string = " ".join(string)
        string = re.sub(r"\s+", " ", string)
        string = string.lower()
        
        normalized.append(string)
        
    return normalized

In [13]:
def filter_stopwords(strings:List[str]) -> List[str]:
    filtered = []
    
    for string in tqdm(strings, desc='Filtering stopwords'):
        words = string.split()
        words = [w for w in words if w not in stopwords.words("russian")]
        string = " ".join(words)
        
        filtered.append(string)
        
    return filtered

In [14]:
def lemmatize_words(strings:List[str]) -> List[str]:
    lemmatized = []
    stem = Mystem()
    
    for string in tqdm(strings, desc='Lemmatizing'):
        lemmas = stem.lemmatize(string)
        lemmas[-1] = lemmas[-1].replace("\n", "")
        string = ''.join(lemmas)
        
        lemmatized.append(string)
        
    return lemmatized

In [15]:
def preprocessing_pipeline(strings:List[str]) -> List[str]:
    strings = normalize_text(strings)
    strings = filter_stopwords(strings)
    strings = lemmatize_words(strings)
    
    return strings

In [16]:
corpus = preprocessing_pipeline(descriptions)

Normalizing texts:   0%|          | 0/17689 [00:00<?, ?it/s]

Filtering stopwords:   0%|          | 0/17689 [00:00<?, ?it/s]

Lemmatizing:   0%|          | 0/17689 [00:00<?, ?it/s]

In [17]:
id_to_text = {i : corpus[i] for i in range(len(corpus))}

In [18]:
with open("../data/movie_descriptions.json", "w") as f:
    f.write(json.dumps(id_to_text))

In [19]:
with open("../data/movie_descriptions.json") as f:
    data = json.loads(f.read())
descriptions = list(data.values())

In [20]:
vectorizer = TfidfVectorizer()
corpus = vectorizer.fit_transform(descriptions)

In [21]:
corpus

<17689x50711 sparse matrix of type '<class 'numpy.float64'>'
	with 1030740 stored elements in Compressed Sparse Row format>

## *Creating labels for descriptions*

In [22]:
labels = one_hot_encoded_genres.apply(lambda x : np.where(x != 0)[0][0], axis=1) #just take first non-zero value of a row as a label

In [23]:
labels

0        2
1        0
2        1
3        0
4        4
        ..
17684    1
17685    0
17686    0
17687    6
17688    2
Length: 17689, dtype: int64

# Fitting and evaluating models

## Splitting data for train and test

In [24]:
x_train, x_test, y_train, y_test = train_test_split(corpus, labels,
                                                    random_state=42,
                                                    test_size=0.1, shuffle=True)

In [25]:
print(x_train.shape, x_test.shape)

(15920, 50711) (1769, 50711)


## KMeans

### *Fitting model*

In [26]:
n_clusters = labels.nunique()
kmeans = KMeans(n_clusters=n_clusters)

In [27]:
kmeans.fit(x_train)

KMeans(n_clusters=12)

### *Evaluating model*

In [28]:
preds = kmeans.predict(x_test)

In [29]:
silhouette_score(preds.reshape(-1, 1), y_test.to_numpy().astype(np.int32))

-0.2411829213324872

## DBScan

### *Fitting model*

In [30]:
dbscan = DBSCAN()

In [31]:
preds = dbscan.fit_predict(corpus)

### *Evaluating model*

In [32]:
silhouette_score(preds.reshape(-1, 1), labels.to_numpy())

-0.5012660284114789

# Indexing

In [33]:
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(corpus.todense())
index.createIndex({'post': 2}, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
******************************************************



In [40]:
ids, distances = index.knnQuery(corpus.todense()[566], k=10)

In [41]:
ids, distances

(array([  566,  1231, 17633, 11187,  2532,  6343,  6335, 16379, 12722,
        10489], dtype=int32),
 array([5.9604645e-08, 6.0702515e-01, 7.9070109e-01, 8.5961443e-01,
        8.6396623e-01, 8.7417018e-01, 8.7441581e-01, 8.7442732e-01,
        8.7451237e-01, 8.7549382e-01], dtype=float32))